# Experiment 04: Oversampling patches

In this experiment we will study how different band configurations affect the training process and end results.

### Experiment variations:

- E03-4: The 4 MS WorldView-2 bands that are also available in GeoEye-1 (MS1 array RGB+NIR1)
 - In this variation we will also validate performance on the GeoEye-1 validation set
- E03-3: Only the 3 RGB bands from WorldView-2
 - Also validated on the GeoEye-1 validation set
 
### The notebook is divided into the following main sections:
1. Imports and configuration parameters
2. Tile generation (sampling of tiles from the satellite images)
3. Tile input pipelines (`tf.dataset` objects reading tiles from disk)
4. Building of models
5. Pretraining with L1 loss
6. Build the full ESRGAN model
7. GAN-training with L1 + Percep + GAN loss
8. Inspection of results

Training history is logged with TensorBoard.

## 1. Imports and configuration parameters

In [1]:
from modules.helpers import *
from modules.tile_generator import *
from modules.matlab_metrics import *
from modules.image_utils import *
from modules.tile_input_pipeline import *
from modules.models import *
from modules.evaluation import *

from modules.logging import *
from modules.train import *

import time

# Check GPUs and enable dynamic GPU memory use:",
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            # Prevent TensorFlow from allocating all memory of all GPUs:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [2]:
### MAIN SETTINGS ###############################################################################################
EXPERIMENT_NAMES = ['e04-8', 'e04-6', 'e04-4', 'e04-3']

# Select experiment variation to be run in THIS notebook:
EXPERIMENT = EXPERIMENT_NAMES[2]

# Turn on and off certain time consuming processes in the notebook:
GENERATE_TILES = False   # This should only be done once in experiment 01. All variations will read from the same
TILE_DENSITY_MAPS = False  # Loops through all tiles and compute density maps of where tiles have been sampled
CALCULATE_STATS = False  # Loops through all tiles and calculate mean and sd. Used for scaling
PRE_BUILD = True          # Step 1 of the training process
PRETRAIN = False          # Step 1 of the training process
GAN_BUILD = True          # Step 1 of the training process
GAN_TRAIN = True         # Step 2 of the training process
PRE_EVALUATE_LAST = False
GAN_EVALUATE_LAST = False
PRE_EVALUATE_HISTORY = False
GAN_EVALUATE_HISTORY = False

# Load metadata dataframe "meta" from repository root. 
# This dataframe keeops track of images and is used and updated throughout the notebook
meta = load_meta_pickle_csv('.', 'metadata_df', from_pickle=True)
#################################################################################################################

### PATHS #######################################################################################################
DATA_PATH = 'data/toulon-laspezia'
DATA_PATH_TILES = 'data/toulon-laspezia-tiles/e04'
DATA_PATH_TILES_P = {'train': DATA_PATH_TILES + '/train', 
                     'val': DATA_PATH_TILES + '/val', 
                     'test': DATA_PATH_TILES + '/test'}
LOGS_DIR = 'logs/' # Path to tensorboard logs and model checkpoint saves
LOGS_EXP_DIR = LOGS_DIR + EXPERIMENT
#################################################################################################################

### TILE GENERATION #############################################################################################
SENSORS_GENERATE = ['WV02', 'GE01']
AREAS_GENERATE = ['La_Spezia', 'Toulon']
meta = subset_by_areas_sensor(meta, areas=AREAS_GENERATE, sensors=SENSORS_GENERATE)
print('Sensors to generate tiles from:', SENSORS_GENERATE)
print('Areas to generate tiles from:', AREAS_GENERATE)

# Count images in partitions (train/val/test):
N_IMAGES_TOTAL = count_images(meta)
N_IMAGES = {'train': count_images_in_partition(meta, 'train'), 
            'val': count_images_in_partition(meta, 'val'), 
            'test': count_images_in_partition(meta, 'test')}
assert N_IMAGES_TOTAL == sum(N_IMAGES.values())  # Verify that different ways of counting adds up
print('Number of images in partitions', N_IMAGES)
print('Total number of images:', N_IMAGES_TOTAL)

TILES_PER_M2 = {'train': 10.0,  ################################# <--- The oversampling of patches
                'val': 2.0, 
                'test': 2.0}

# Settings for whether to send every tile generated through a sea and cloud classifier
# This is useful if images consist of a lot of sea and clouds and you want to reduce the number of tiles
# with such monotone and less meaningful content. Classifier is trained on 2500 labeled tiles of various sizes
# where only tiles COMPLETELY covered by sea and/or clouds have been labelled "cloud/sea". 
# Validation accuracy around 0.95
CLOUD_SEA_REMOVAL = True
CLOUD_SEA_WEIGHTS_PATH = 'models/cloud-sea-classifier/cloudsea-effb0-augm-bicubic-pan-0.0005--200-0.127841.h5'
# Cutoff at inference time. Tiles with (quasi)-prob higher than cutoff will be classified as cloud and or sea:
CLOUD_SEA_PRED_CUTOFF = 0.95
# Setting to keep a certain proportion of cloud/sea tiles through the filter:
CLOUD_SEA_KEEP_RATE = 0.10

# GE01 images has some slight variations in resolution 0.5 +-0.05 m per pixel while WV02 is fixed at 0.5m
# Setting this to True will resize to as close as possible to 0.5m
# Not used in this notebook, but function is ready for use in module tile_generator.py
RESIZE_TO_PIXEL_SIZE = False
if RESIZE_TO_PIXEL_SIZE:
    RESIZE_RESAMPLING_METHOD = 'nearest'  # 'nearest', 'bicubic', 'bilinear'
    NEW_PIXEL_SIZE_PAN = 0.5
    RESIZE_DIR = DATA_PATH + '-resized'
#################################################################################################################

### SENSORS AND AREA EXPERIMENT SELECTION #######################################################################
# Sensors used in which experiment variation
SENSORS_EXP = {'e04-8': {'train': 'WV02', 'val': ['WV02'], 'test': ['WV02']}, 
               'e04-6': {'train': 'WV02', 'val': ['WV02'], 'test': ['WV02']}, 
               'e04-4': {'train': 'WV02', 'val': ['WV02', 'GE01'], 'test': ['WV02', 'GE01']}, 
               'e04-3': {'train': 'WV02', 'val': ['WV02', 'GE01'], 'test': ['WV02', 'GE01']}}
SENSORS = SENSORS_EXP[EXPERIMENT]

# Areas used in which experiment variation
AREAS_EXP = {'e04-8': {'train': AREAS_GENERATE, 'val': AREAS_GENERATE, 'test': AREAS_GENERATE}, 
             'e04-6': {'train': AREAS_GENERATE, 'val': AREAS_GENERATE, 'test': AREAS_GENERATE}, 
             'e04-4': {'train': AREAS_GENERATE, 'val': AREAS_GENERATE, 'test': AREAS_GENERATE}, 
             'e04-3': {'train': AREAS_GENERATE, 'val': AREAS_GENERATE, 'test': AREAS_GENERATE}}
AREAS = AREAS_EXP[EXPERIMENT]
#################################################################################################################

### TILE DIMENSIONS #############################################################################################
# Note larger size of val and test. This is needed for sensible calculation of Ma, NIQE and PI calculation
SR_FACTOR = 4
MS_SIZE = {'train': 32, 'val': 128, 'test': 128}
PAN_SIZE = {'train': MS_SIZE['train'] * SR_FACTOR, 
            'val': MS_SIZE['val'] * SR_FACTOR, 
            'test': MS_SIZE['test'] * SR_FACTOR}
print('MS (LR) tile size:', MS_SIZE)
print('PAN (HR) tile size:', PAN_SIZE)
print('SR factor:', SR_FACTOR)
#################################################################################################################

### BAND (CHANNEL) CONFIGURATIONS ###############################################################################
# This is the essence of experiment 01
# Selection of bands is done in the tile input pipeline

# Selecting bands from the 8 bands of WV02:
WV02_FULL_BAND_CONFIG = get_sensor_bands('WV02', meta)
WV02_EXP_BAND_CONFIGS = {'e04-8': WV02_FULL_BAND_CONFIG,                          # 8 (all) bands
                         'e04-6': {k:v for (k,v) in WV02_FULL_BAND_CONFIG.items()  # 6 bands (BGYR+RE+NIR)
                                   if k not in ['Coastal', 'NIR2']}, 
                         'e04-4': {k:v for (k,v) in WV02_FULL_BAND_CONFIG.items()  # 4 bands (BGR+NIR)
                                   if k in ['Blue', 'Green', 'Red', 'NIR']},
                         'e04-3': {k:v for (k,v) in WV02_FULL_BAND_CONFIG.items()  # 3 bands (BGR)
                                   if k in ['Blue', 'Green', 'Red']}}
MS_BANDS_WV02_CONFIG = WV02_EXP_BAND_CONFIGS[EXPERIMENT]
if EXPERIMENT == 'e04-8':
    # We set this to 'all' in order to not pass e01-8 tiles through a band selection function (no reason to)
    MS_BANDS_WV02_IDXS = 'all' 
else:
    # For the other experiment variations we need lists of indices of the bands to be selected
    MS_BANDS_WV02_IDXS = list(MS_BANDS_WV02_CONFIG.values())

N_MS_BANDS = len(MS_BANDS_WV02_CONFIG.values()) # The number of MS bands in this experiment variation

# Selecting bands from the 4 bands of GE01:
GE01_FULL_BAND_CONFIG = get_sensor_bands('GE01', meta)                            
GE01_EXP_BAND_CONFIGS = {'e04-8': {None: None},                                   # not enough bands in GE01
                         'e04-6': {None: None},                                    # not enough bands in GE01
                         'e04-4': GE01_FULL_BAND_CONFIG,                           # 4 (all) bands (BGR+NIR)
                         'e04-3': {k:v for (k,v) in GE01_FULL_BAND_CONFIG.items()  # 3 bands (BGR)
                                   if k not in ['NIR']}}
MS_BANDS_GE01_CONFIG = GE01_EXP_BAND_CONFIGS[EXPERIMENT]
if EXPERIMENT == 'e04-4':
    MS_BANDS_GE01_IDXS = 'all'
else:
    MS_BANDS_GE01_IDXS = list(MS_BANDS_GE01_CONFIG.values())
print('MS (LR) Band Config WV02:', MS_BANDS_WV02_CONFIG)
print('MS (LR) Band Config GE01:', MS_BANDS_GE01_CONFIG)

N_PAN_BANDS = 1 # Obviously only 1 panchromatic band
#################################################################################################################

### MODEL PARAMETERS ############################################################################################
BATCH_SIZE = {'train': 16, 'val': 8, 'test': 8}
print('Batch sizes:', BATCH_SIZE)

# RRDB Generator Model parameters 
N_BLOCKS = 16 # Deeper means potential to capture more complex relationships, at the cost of training time
N_FILTERS = 64 # Baseline setting that is not tinkered with in this repository
#################################################################################################################

### PRETRAINING SETTINGS ########################################################################################
PRE_EPOCHS = 400
PRE_TRAIN_STEPS = 1000  # per epoch
PRE_VAL_STEPS = 0     # per epoch
print('Pretraining - Total steps:', PRE_EPOCHS * PRE_TRAIN_STEPS)

# Number of batches to save every epoch in TensorBoard
TRAIN_N_BATCHES_SAVE = 1
VAL_N_BATCHES_SAVE = 1

# Optimizer settings:
PRETRAIN_LOSS = 'l1'    # Official
PRETRAIN_LR = 5e-5      # Tuned and found stable for this particular experiment
#PRETRAIN_LR = 0.0002   # Official
PRETRAIN_BETA_1 = 0.9   # Official
PRETRAIN_BETA_2 = 0.999 # Official
# Note: Official implementation also uses stepwise learning rate scheduler. 
# This is avoided here as it is deemed not central to the experiment to "squeeze" out last performance and it 
# complicates comparisons between experiment variations
#################################################################################################################

### GAN TRAINING SETTINGS #######################################################################################
GAN_EPOCHS = 400
GAN_TRAIN_STEPS = 1000
GAN_VAL_STEPS = 0
# Proportion of val batches that will go through ma and niqe metric calculation
# MA_NIQE_PROPORTION = 0.04  # The calculation is very time consuming
MA_NIQE_PROPORTION = 1  # The calculation is very time consuming
print('GAN training - Total steps:', GAN_EPOCHS * GAN_TRAIN_STEPS)

# Weights for each loss in the composite loss function
G_LOSS_PIXEL_W = 0.01       # Official
G_LOSS_PERCEP_W = 1.0       # Official
G_LOSS_GENERATOR_W = 0.005  # Official

# Optimizer settings:
#GAN_G_LR = 1e-4 # Official
#GAN_D_LR = 1e-4 # Official
GAN_G_LR = 2e-5
GAN_D_LR = 2e-5
G_BETA_1, D_BETA_1 = 0.9, 0.9      # Official
G_BETA_2, D_BETA_2 = 0.999, 0.999  # Official
# Note: Official implementation also uses stepwise learning rate scheduler. 
# This is avoided here as it is deemed not central to the experiment to "squeeze" out last performance and it 
# complicates comparisons between experiment variations

# Path to the pretraining weights that is the starting point of GAN training:
PRETRAIN_WEIGHTS_DIRS = {'e04-4': LOGS_EXP_DIR + '/models/' + 'e04-4-pre_20210314-030336/'
                        }
PRETRAIN_WEIGHTS_DIR = PRETRAIN_WEIGHTS_DIRS[EXPERIMENT]
PRETRAIN_WEIGHTS_PATH = PRETRAIN_WEIGHTS_DIR + EXPERIMENT + '-pre-400.h5'

# Path to the gan-training weights that will be 
GAN_WEIGHTS_DIRS = {'e04-4': LOGS_EXP_DIR + '/models/' + 'e04-4-gan_20210315-230940/'
                   }
GAN_WEIGHTS_DIR = GAN_WEIGHTS_DIRS[EXPERIMENT]
GAN_WEIGHTS_PATH = GAN_WEIGHTS_DIR + EXPERIMENT + '-gan-G-399.h5'
#################################################################################################################

### MATLAB METRICS ##############################################################################################
# Calculate Ma, NIQE and Perceptual Index (PI) metrics on the validation set(s) during GAN training:
# PI was metric used in PIRM2018 competition https://github.com/roimehrez/PIRM2018
METRIC_MA = False
METRIC_NIQE = False
if METRIC_MA and METRIC_NIQE:
    METRIC_PI = True
else:
    METRIC_PI = False

# The number of pixels to be shaved off the border of the tile before calculating Ma/NIQE/PI (ignore border effects)
SHAVE_WIDTH = 4 # Official (as used in PIRM2018 evaluation)
# Ma/NIQE/PI calculation is done with official matlab repositories through MATLAB Engine API for Python
MATLAB_PATH = 'modules/matlab' # path to repositories
#################################################################################################################

### EVALUTAION ##################################################################################################
if PRE_EVALUATE_LAST or GAN_EVALUATE_LAST:
    METRIC_MA = False
    METRIC_NIQE = True
    if METRIC_MA and METRIC_NIQE:
        METRIC_PI = True
    else:
        METRIC_PI = False
        
if PRE_EVALUATE_HISTORY and GAN_EVALUATE_HISTORY:
    raise ValueError

EVAL_STEPS_PER_EPOCH = 2000
EVAL_N_EPOCHS = 400
EVAL_SENSOR = 'WV02'
EVAL_PER_IMAGE = True
    
if PRE_EVALUATE_HISTORY:
    EVAL_WEIGHTS_DIR = PRETRAIN_WEIGHTS_DIR
    EVAL_FIRST_STEP = 1
    EVAL_PREFIX = EXPERIMENT + '-pre-'
elif GAN_EVALUATE_HISTORY:
    EVAL_WEIGHTS_DIR = GAN_WEIGHTS_DIR
    EVAL_FIRST_STEP = 0
    EVAL_PREFIX = EXPERIMENT + '-gan-'
    
print('MATLAB Metrics:')
print('Ma:', METRIC_MA)
print('NIQE:', METRIC_NIQE)
print('Perceptual Index (PI):', METRIC_PI)

Sensors to generate tiles from: ['WV02', 'GE01']
Areas to generate tiles from: ['La_Spezia', 'Toulon']
Number of images in partitions {'train': 22, 'val': 19, 'test': 21}
Total number of images: 62
MS (LR) tile size: {'train': 32, 'val': 128, 'test': 128}
PAN (HR) tile size: {'train': 128, 'val': 512, 'test': 512}
SR factor: 4
MS (LR) Band Config WV02: {'Blue': 1, 'Green': 2, 'Red': 4, 'NIR': 6}
MS (LR) Band Config GE01: {'Blue': 0, 'Green': 1, 'Red': 2, 'NIR': 3}
Batch sizes: {'train': 16, 'val': 8, 'test': 8}
Pretraining - Total steps: 400000
GAN training - Total steps: 400000
MATLAB Metrics:
Ma: False
NIQE: False
Perceptual Index (PI): False


## 2. Tile generation

### 2.1 Image resizing

Function `resize_sat_img_to_new_pixel_size` available in `modules.tile_generator`. Not used in this notebook

### 2.2 Tile allocation

We allocate `n_tiles` to each satellite image in proportion to the area covered by the satellite image. We adjust `n_tiles` by the argument `tiles_per_m2`. If `tiles_per_m2=1.0` then `n_tiles` is set deterministically to a value so that a square meter of satellite image is expected to be covered by `1.0` tile.

In [3]:
if GENERATE_TILES:
    meta = allocate_tiles_by_expected(meta, 
                                      override_pan_pixel_size=RESIZE_TO_PIXEL_SIZE,
                                      by_partition=True, 
                                      tiles_per_m2_train_val_test=(TILES_PER_M2['train'], 
                                                                   TILES_PER_M2['val'], 
                                                                   TILES_PER_M2['test']),
                                      pan_tile_size_train_val_test=(PAN_SIZE['train'], 
                                                                    PAN_SIZE['val'], 
                                                                    PAN_SIZE['test']),
                                      new_column_name='n_tiles')
else:
    # Load meta dataframe that was updated at tile generation time
    meta = load_meta_pickle_csv(DATA_PATH_TILES, 'metadata_tile_allocation', from_pickle=True)

n_tiles = {'train': count_tiles_in_partition(meta, 'train'),
           'val': count_tiles_in_partition(meta, 'val'), 
           'test':  count_tiles_in_partition(meta, 'test')}
n_tiles_total = count_tiles(meta)
assert n_tiles_total == sum(n_tiles.values())
print('Number of tiles per partition:')
print(n_tiles)
print('Total number of tiles:', n_tiles_total)

Number of tiles per partition:
{'train': 645320, 'val': 8050, 'test': 9315}
Total number of tiles: 662685


### 2.3 Tile generation to disk

In [4]:
if GENERATE_TILES:
    meta = generate_all_tiles(meta, 
                              save_dir=DATA_PATH_TILES, 
                              sr_factor=SR_FACTOR, 
                              by_partition=True,
                              ms_tile_size_train_val_test=(MS_SIZE['train'], MS_SIZE['val'], MS_SIZE['test']), 
                              cloud_sea_removal=CLOUD_SEA_REMOVAL, 
                              cloud_sea_weights_path=CLOUD_SEA_WEIGHTS_PATH, 
                              cloud_sea_pred_cutoff=CLOUD_SEA_PRED_CUTOFF,
                              cloud_sea_keep_rate=CLOUD_SEA_KEEP_RATE,
                              save_meta_to_disk=True)

In [5]:
if TILE_DENSITY_MAPS:
    for row in meta.iterrows():
        img_uid = row[0]
        density = tile_density_map(DATA_PATH_TILES, 
                                   row[1], 
                                   pan_or_ms='pan',
                                   density_dtype='uint8',
                                   write_to_disk=True,
                                   write_dir=DATA_PATH_TILES + '/density-maps', 
                                   write_filename=img_uid)
    # Plot last density
    plt.imshow(density)

In [6]:
if CALCULATE_STATS:
    train_tiles_mean, train_tiles_sd = mean_sd_of_train_tiles(DATA_PATH_TILES, 
                                                              sample_proportion=1.0, 
                                                              write_json=True)
else:
    train_tiles_mean, train_tiles_sd = read_mean_sd_json(DATA_PATH_TILES)

Loaded mean 340.2 and sd 128.5 from json file @ data/toulon-laspezia-tiles/e04/train_mean_sd.json


## 3. Data input pipeline from disk

### 3.1 Training set

In [7]:
SHUFFLE_BUFFER_SIZE = {'train': 130000,  # 100
                       'val': 8000,  # 100
                       'test': 8000}  # 100

train_val_test = 'train'
sensor = SENSORS[train_val_test]
ds_train = {sensor: GeotiffDataset(tiles_path=DATA_PATH_TILES_P[train_val_test], 
                                   batch_size=BATCH_SIZE[train_val_test], 
                                   ms_tile_shape=(MS_SIZE[train_val_test], MS_SIZE[train_val_test], N_MS_BANDS), 
                                   pan_tile_shape=(PAN_SIZE[train_val_test], PAN_SIZE[train_val_test], N_PAN_BANDS),
                                   sensor=sensor,
                                   band_selection=MS_BANDS_WV02_IDXS, 
                                   mean_correction=train_tiles_mean,
                                   cache_memory=True,
                                   cache_file=str(DATA_PATH_TILES + '/ds_' + EXPERIMENT + '-'
                                                    + train_val_test + '-' + sensor + '_cache'), 
                                   repeat=True, 
                                   shuffle=True, 
                                   shuffle_buffer_size=SHUFFLE_BUFFER_SIZE[train_val_test])
           }
# Getting the scaled output range from the scaler. Needed to calculate PSNR and SSIM:
scaled_range = ds_train[sensor].get_scaler_output_range(print_ranges=True)

# Returning the actual tf.data.dataset object:
ds_train[sensor] = ds_train[sensor].get_dataset()
print(ds_train.keys())

Scaler ranges:
Input (uint) min, max: 0 2047
Input (uint) range: 2048
Output (float) range 1.199917335793007
Output (float) min, max: -0.19933143865638941 1.0
dict_keys(['WV02'])


### 3.2 Validation set

In [8]:
# Validation set can have several sensors and is organized in a dictionary
# structure: ds_val = {sensor: dataset} ... ex: ds_val = {'WV02': dataset_with_only_WV02_images}
train_val_test = 'val'
ds_val = {}
for sensor in SENSORS[train_val_test]:
    if sensor == 'WV02':
        band_indices = MS_BANDS_WV02_IDXS
    elif sensor == 'GE01':
        band_indices = MS_BANDS_GE01_IDXS
    ds_val[sensor] = GeotiffDataset(tiles_path=DATA_PATH_TILES_P[train_val_test], 
                                    batch_size=BATCH_SIZE[train_val_test], 
                                    ms_tile_shape=(MS_SIZE[train_val_test], MS_SIZE[train_val_test], N_MS_BANDS), 
                                    pan_tile_shape=(PAN_SIZE[train_val_test], PAN_SIZE[train_val_test], N_PAN_BANDS),
                                    sensor=sensor,
                                    band_selection=band_indices, 
                                    mean_correction=train_tiles_mean,
                                    cache_memory=True,
                                    cache_file=str(DATA_PATH_TILES + '/ds_' + EXPERIMENT + '-'
                                                   + train_val_test + '-' + sensor + '_cache'), 
                                    repeat=True, 
                                    shuffle=True, 
                                    shuffle_buffer_size=SHUFFLE_BUFFER_SIZE[train_val_test])
    ds_val[sensor] = ds_val[sensor].get_dataset()
print(ds_val.keys())

dict_keys(['WV02', 'GE01'])


## 4. Build preliminary models

### 4.1 Bicubic baseline model

In [9]:
bicubic = build_deterministic_sr_model(upsample_factor=SR_FACTOR,
                                       resize_method='bicubic',
                                       loss='mean_absolute_error',
                                       metrics=('PSNR', 'SSIM'),
                                       scaled_range=scaled_range)

### 4.2 ESRGAN Generator model (pretrain version)

In [10]:
if PRE_BUILD:
    pretrain_model =  build_generator(pretrain_or_gan='pretrain', 
                                      pretrain_learning_rate=PRETRAIN_LR, 
                                      pretrain_loss_l1_l2=PRETRAIN_LOSS,
                                      pretrain_beta_1=PRETRAIN_BETA_1, 
                                      pretrain_beta_2=PRETRAIN_BETA_2, 
                                      pretrain_metrics=('PSNR', 'SSIM'),
                                      scaled_range=scaled_range, 
                                      n_channels_in=N_MS_BANDS, 
                                      n_channels_out=N_PAN_BANDS, 
                                      height_width_in=None,  # None will make network image size agnostic
                                      n_filters=N_FILTERS, 
                                      n_blocks=N_BLOCKS)
    # pretrain_model.summary()

## 5. Pretraining with L1 loss

In [11]:
if PRETRAIN:
    history = pretrain_esrgan(generator=pretrain_model,
                              ds_train_dict=ds_train,
                              epochs=PRE_EPOCHS,
                              steps_per_epoch=PRE_TRAIN_STEPS,
                              initial_epoch=0,
                              validate=True,
                              ds_val_dict=ds_val,
                              val_steps=PRE_VAL_STEPS,
                              model_name=EXPERIMENT + '-pre',
                              tag=EXPERIMENT,
                              log_tensorboard=True,
                              tensorboard_logs_dir=LOGS_EXP_DIR + '/tb',
                              save_models=True,
                              models_save_dir=LOGS_EXP_DIR + '/models',
                              save_weights_only=True,
                              log_train_images=True,
                              n_train_image_batches=TRAIN_N_BATCHES_SAVE,
                              log_val_images=True,
                              n_val_image_batches=VAL_N_BATCHES_SAVE)

## 6. Build the full ESRGAN Model

In [12]:
if GAN_BUILD:
    gan_model = build_esrgan_model(PRETRAIN_WEIGHTS_PATH,
                                   n_channels_in=N_MS_BANDS, 
                                   n_channels_out=N_PAN_BANDS, 
                                   n_filters=N_FILTERS, 
                                   n_blocks=N_BLOCKS, 
                                   pan_shape=(PAN_SIZE['train'], PAN_SIZE['train'], N_PAN_BANDS),
                                   G_lr=GAN_G_LR, 
                                   D_lr=GAN_D_LR, 
                                   G_beta_1=G_BETA_1, 
                                   G_beta_2=G_BETA_2, 
                                   D_beta_1=D_BETA_1, 
                                   D_beta_2=D_BETA_2,
                                   G_loss_pixel_w=G_LOSS_PIXEL_W, 
                                   G_loss_pixel_l1_l2='l1',
                                   G_loss_percep_w=G_LOSS_PERCEP_W, 
                                   G_loss_percep_l1_l2='l1', 
                                   G_loss_percep_layer=54,
                                   G_loss_percep_before_act=True,
                                   G_loss_generator_w=G_LOSS_GENERATOR_W,
                                   metric_reg=False, 
                                   metric_ma=METRIC_MA, 
                                   metric_niqe=METRIC_NIQE, 
                                   ma_niqe_proportion=MA_NIQE_PROPORTION,
                                   matlab_wd_path='modules/matlab',
                                   scale_mean=train_tiles_mean, 
                                   scaled_range=scaled_range, 
                                   shave_width=SHAVE_WIDTH)

## 7. GAN training

In [13]:
if GAN_TRAIN:
    history = gan_train_esrgan(esrgan_model=gan_model,
                               ds_train_dict=ds_train,
                               epochs=GAN_EPOCHS,
                               steps_per_epoch=GAN_TRAIN_STEPS,
                               initial_epoch=0,
                               validate=True,
                               ds_val_dict=ds_val,
                               val_steps=GAN_VAL_STEPS,
                               model_name=EXPERIMENT + '-gan',
                               tag=EXPERIMENT,
                               log_tensorboard=True,
                               tensorboard_logs_dir=LOGS_EXP_DIR + '/tb',
                               save_models=True,
                               models_save_dir=LOGS_EXP_DIR + '/models',
                               save_weights_only=True,
                               log_train_images=True,
                               n_train_image_batches=TRAIN_N_BATCHES_SAVE,
                               log_val_images=True,
                               n_val_image_batches=VAL_N_BATCHES_SAVE)

16 images from train-WV02 will be logged at each epoch
8 images from val-WV02 will be logged at each epoch
8 images from val-GE01 will be logged at each epoch
Callbacks: [<modules.logging.SimpleEsrganCheckpoint object at 0x0000027401D962C8>, <tensorflow.python.keras.callbacks.TensorBoard object at 0x0000027403DD7C48>, <modules.logging.MultipleValSetsCallback object at 0x0000027401D91648>, <modules.logging.LrHrSrImageCallback object at 0x0000027401D96408>, <modules.logging.LrHrSrImageCallback object at 0x0000027403CC5308>]
Epoch 1/400
1000/1000 [==============================] - 472s 472ms/step - G_pixel_loss: 1.0984e-04 - G_perceptual_loss: 1.9620 - G_generator_loss: 0.0217 - G_loss_total: 1.9838 - D_loss_total: 0.1519 - PSNR: 38.9017 - SSIM: 0.9127
Epoch 2/400
1000/1000 [==============================] - 466s 466ms/step - G_pixel_loss: 1.1206e-04 - G_perceptual_loss: 1.8890 - G_generator_loss: 0.0381 - G_loss_total: 1.9272 - D_loss_total: 0.0355 - PSNR: 38.3474 - SSIM: 0.9123
Epoch 3/

1000/1000 [==============================] - 469s 469ms/step - G_pixel_loss: 1.1170e-04 - G_perceptual_loss: 1.7977 - G_generator_loss: 0.0423 - G_loss_total: 1.8401 - D_loss_total: 0.0634 - PSNR: 38.5541 - SSIM: 0.9033
Epoch 34/400
1000/1000 [==============================] - 465s 465ms/step - G_pixel_loss: 1.1048e-04 - G_perceptual_loss: 1.7695 - G_generator_loss: 0.0429 - G_loss_total: 1.8126 - D_loss_total: 0.0743 - PSNR: 38.6998 - SSIM: 0.9045
Epoch 35/400
1000/1000 [==============================] - 465s 465ms/step - G_pixel_loss: 1.1202e-04 - G_perceptual_loss: 1.7839 - G_generator_loss: 0.0396 - G_loss_total: 1.8236 - D_loss_total: 0.0611 - PSNR: 38.5354 - SSIM: 0.9037
Epoch 36/400
1000/1000 [==============================] - 465s 465ms/step - G_pixel_loss: 1.1185e-04 - G_perceptual_loss: 1.7807 - G_generator_loss: 0.0432 - G_loss_total: 1.8241 - D_loss_total: 0.0692 - PSNR: 38.5794 - SSIM: 0.9033
Epoch 37/400
1000/1000 [==============================] - 465s 465ms/step - G_pix

1000/1000 [==============================] - 467s 467ms/step - G_pixel_loss: 1.1015e-04 - G_perceptual_loss: 1.7387 - G_generator_loss: 0.0402 - G_loss_total: 1.7789 - D_loss_total: 0.0664 - PSNR: 38.6484 - SSIM: 0.9044
Epoch 69/400
1000/1000 [==============================] - 469s 469ms/step - G_pixel_loss: 1.1077e-04 - G_perceptual_loss: 1.7387 - G_generator_loss: 0.0355 - G_loss_total: 1.7743 - D_loss_total: 0.0834 - PSNR: 38.6209 - SSIM: 0.9033
Epoch 70/400
1000/1000 [==============================] - 468s 468ms/step - G_pixel_loss: 1.1138e-04 - G_perceptual_loss: 1.7343 - G_generator_loss: 0.0443 - G_loss_total: 1.7787 - D_loss_total: 0.0511 - PSNR: 38.5608 - SSIM: 0.9033
Epoch 71/400
1000/1000 [==============================] - 468s 468ms/step - G_pixel_loss: 1.0926e-04 - G_perceptual_loss: 1.7313 - G_generator_loss: 0.0456 - G_loss_total: 1.7770 - D_loss_total: 0.0501 - PSNR: 38.7221 - SSIM: 0.9043
Epoch 72/400
1000/1000 [==============================] - 471s 471ms/step - G_pix

1000/1000 [==============================] - 466s 466ms/step - G_pixel_loss: 1.1008e-04 - G_perceptual_loss: 1.7064 - G_generator_loss: 0.0456 - G_loss_total: 1.7521 - D_loss_total: 0.0420 - PSNR: 38.6706 - SSIM: 0.9040
Epoch 104/400
1000/1000 [==============================] - 466s 466ms/step - G_pixel_loss: 1.1111e-04 - G_perceptual_loss: 1.6974 - G_generator_loss: 0.0440 - G_loss_total: 1.7415 - D_loss_total: 0.0460 - PSNR: 38.6556 - SSIM: 0.9030
Epoch 105/400
1000/1000 [==============================] - 467s 467ms/step - G_pixel_loss: 1.1097e-04 - G_perceptual_loss: 1.7006 - G_generator_loss: 0.0461 - G_loss_total: 1.7468 - D_loss_total: 0.0481 - PSNR: 38.6499 - SSIM: 0.9023
Epoch 106/400
1000/1000 [==============================] - 465s 465ms/step - G_pixel_loss: 1.1109e-04 - G_perceptual_loss: 1.6929 - G_generator_loss: 0.0476 - G_loss_total: 1.7406 - D_loss_total: 0.0522 - PSNR: 38.6377 - SSIM: 0.9025
Epoch 107/400
1000/1000 [==============================] - 465s 465ms/step - G

1000/1000 [==============================] - 466s 466ms/step - G_pixel_loss: 1.1193e-04 - G_perceptual_loss: 1.6774 - G_generator_loss: 0.0536 - G_loss_total: 1.7311 - D_loss_total: 0.0282 - PSNR: 38.5267 - SSIM: 0.9028
Epoch 139/400
1000/1000 [==============================] - 466s 466ms/step - G_pixel_loss: 1.1216e-04 - G_perceptual_loss: 1.6845 - G_generator_loss: 0.0499 - G_loss_total: 1.7345 - D_loss_total: 0.0534 - PSNR: 38.5145 - SSIM: 0.9017
Epoch 140/400
1000/1000 [==============================] - 466s 466ms/step - G_pixel_loss: 1.1112e-04 - G_perceptual_loss: 1.6677 - G_generator_loss: 0.0494 - G_loss_total: 1.7172 - D_loss_total: 0.0508 - PSNR: 38.5957 - SSIM: 0.9026
Epoch 141/400
1000/1000 [==============================] - 468s 468ms/step - G_pixel_loss: 1.1093e-04 - G_perceptual_loss: 1.6740 - G_generator_loss: 0.0558 - G_loss_total: 1.7299 - D_loss_total: 0.0243 - PSNR: 38.6204 - SSIM: 0.9024
Epoch 142/400
1000/1000 [==============================] - 467s 467ms/step - G

...

1000/1000 [==============================] - 466s 466ms/step - G_pixel_loss: 1.1100e-04 - G_perceptual_loss: 1.6063 - G_generator_loss: 0.0576 - G_loss_total: 1.6641 - D_loss_total: 0.0395 - PSNR: 38.6353 - SSIM: 0.9027
Epoch 244/400
1000/1000 [==============================] - 465s 465ms/step - G_pixel_loss: 1.1077e-04 - G_perceptual_loss: 1.6088 - G_generator_loss: 0.0610 - G_loss_total: 1.6699 - D_loss_total: 0.0260 - PSNR: 38.5601 - SSIM: 0.9026
Epoch 245/400
1000/1000 [==============================] - 466s 466ms/step - G_pixel_loss: 1.1091e-04 - G_perceptual_loss: 1.6089 - G_generator_loss: 0.0652 - G_loss_total: 1.6742 - D_loss_total: 0.0112 - PSNR: 38.6534 - SSIM: 0.9014
Epoch 246/400
1000/1000 [==============================] - 465s 465ms/step - G_pixel_loss: 1.0998e-04 - G_perceptual_loss: 1.6033 - G_generator_loss: 0.0581 - G_loss_total: 1.6615 - D_loss_total: 0.0322 - PSNR: 38.7309 - SSIM: 0.9030
Epoch 247/400
1000/1000 [==============================] - 465s 465ms/step - G

1000/1000 [==============================] - 467s 467ms/step - G_pixel_loss: 1.1099e-04 - G_perceptual_loss: 1.5996 - G_generator_loss: 0.0620 - G_loss_total: 1.6617 - D_loss_total: 0.0358 - PSNR: 38.6587 - SSIM: 0.9035
Epoch 279/400
1000/1000 [==============================] - 467s 467ms/step - G_pixel_loss: 1.1244e-04 - G_perceptual_loss: 1.6079 - G_generator_loss: 0.0624 - G_loss_total: 1.6704 - D_loss_total: 0.0278 - PSNR: 38.5240 - SSIM: 0.9011
Epoch 280/400
1000/1000 [==============================] - 467s 467ms/step - G_pixel_loss: 1.1189e-04 - G_perceptual_loss: 1.5998 - G_generator_loss: 0.0587 - G_loss_total: 1.6586 - D_loss_total: 0.0271 - PSNR: 38.5305 - SSIM: 0.9021
Epoch 281/400
1000/1000 [==============================] - 467s 467ms/step - G_pixel_loss: 1.0957e-04 - G_perceptual_loss: 1.5860 - G_generator_loss: 0.0531 - G_loss_total: 1.6392 - D_loss_total: 0.0353 - PSNR: 38.7645 - SSIM: 0.9032
Epoch 282/400
1000/1000 [==============================] - 468s 468ms/step - G

1000/1000 [==============================] - 465s 465ms/step - G_pixel_loss: 1.1082e-04 - G_perceptual_loss: 1.5916 - G_generator_loss: 0.0636 - G_loss_total: 1.6553 - D_loss_total: 0.0196 - PSNR: 38.6783 - SSIM: 0.9023
Epoch 314/400
1000/1000 [==============================] - 465s 465ms/step - G_pixel_loss: 1.1165e-04 - G_perceptual_loss: 1.5850 - G_generator_loss: 0.0637 - G_loss_total: 1.6488 - D_loss_total: 0.0318 - PSNR: 38.5678 - SSIM: 0.9017
Epoch 315/400
1000/1000 [==============================] - 465s 465ms/step - G_pixel_loss: 1.1143e-04 - G_perceptual_loss: 1.5818 - G_generator_loss: 0.0633 - G_loss_total: 1.6452 - D_loss_total: 0.0280 - PSNR: 38.6425 - SSIM: 0.9029
Epoch 316/400
1000/1000 [==============================] - 466s 466ms/step - G_pixel_loss: 1.1091e-04 - G_perceptual_loss: 1.5836 - G_generator_loss: 0.0697 - G_loss_total: 1.6534 - D_loss_total: 0.0188 - PSNR: 38.6411 - SSIM: 0.9028
Epoch 317/400
1000/1000 [==============================] - 466s 466ms/step - G

1000/1000 [==============================] - 466s 466ms/step - G_pixel_loss: 1.1125e-04 - G_perceptual_loss: 1.5690 - G_generator_loss: 0.0619 - G_loss_total: 1.6311 - D_loss_total: 0.0269 - PSNR: 38.6398 - SSIM: 0.9013
Epoch 349/400
1000/1000 [==============================] - 466s 466ms/step - G_pixel_loss: 1.1093e-04 - G_perceptual_loss: 1.5768 - G_generator_loss: 0.0716 - G_loss_total: 1.6485 - D_loss_total: 0.0166 - PSNR: 38.5977 - SSIM: 0.9023
Epoch 350/400
1000/1000 [==============================] - 466s 466ms/step - G_pixel_loss: 1.1141e-04 - G_perceptual_loss: 1.5644 - G_generator_loss: 0.0739 - G_loss_total: 1.6383 - D_loss_total: 0.0263 - PSNR: 38.6342 - SSIM: 0.9025
Epoch 351/400
1000/1000 [==============================] - 466s 466ms/step - G_pixel_loss: 1.1194e-04 - G_perceptual_loss: 1.5656 - G_generator_loss: 0.0630 - G_loss_total: 1.6287 - D_loss_total: 0.0258 - PSNR: 38.6170 - SSIM: 0.9019
Epoch 352/400
1000/1000 [==============================] - 465s 465ms/step - G

1000/1000 [==============================] - 466s 466ms/step - G_pixel_loss: 1.1089e-04 - G_perceptual_loss: 1.5579 - G_generator_loss: 0.0615 - G_loss_total: 1.6195 - D_loss_total: 0.0119 - PSNR: 38.7081 - SSIM: 0.9018
Epoch 384/400
1000/1000 [==============================] - 466s 466ms/step - G_pixel_loss: 1.1162e-04 - G_perceptual_loss: 1.5594 - G_generator_loss: 0.0599 - G_loss_total: 1.6194 - D_loss_total: 0.0279 - PSNR: 38.5889 - SSIM: 0.9020
Epoch 385/400
1000/1000 [==============================] - 466s 466ms/step - G_pixel_loss: 1.1136e-04 - G_perceptual_loss: 1.5530 - G_generator_loss: 0.0583 - G_loss_total: 1.6114 - D_loss_total: 0.0290 - PSNR: 38.6243 - SSIM: 0.9020
Epoch 386/400
1000/1000 [==============================] - 466s 466ms/step - G_pixel_loss: 1.1166e-04 - G_perceptual_loss: 1.5613 - G_generator_loss: 0.0611 - G_loss_total: 1.6225 - D_loss_total: 0.0172 - PSNR: 38.5377 - SSIM: 0.9019
Epoch 387/400
1000/1000 [==============================] - 466s 466ms/step - G

## 8. Evaluation

### 8.1 Data input pipelines for final evaluation

The pipeline is modified to include the file paths of the tiles/patches so that it is possible to log performance metrics for individual files and by extension for individual satellite images.

#### 8.1.1 Validation set

In [14]:
# Validation set can have several sensors and is organized in a dictionary
# structure: ds_val = {sensor: dataset} ... ex: ds_val = {'WV02': dataset_with_only_WV02_images}
train_val_test = 'val'
ds_val = {}
for sensor in SENSORS[train_val_test]:
    if sensor == 'WV02':
        band_indices = MS_BANDS_WV02_IDXS
    elif sensor == 'GE01':
        band_indices = MS_BANDS_GE01_IDXS
    ds_val[sensor] = GeotiffDataset(tiles_path=DATA_PATH_TILES_P[train_val_test], 
                                    batch_size=BATCH_SIZE[train_val_test], 
                                    ms_tile_shape=(MS_SIZE[train_val_test], MS_SIZE[train_val_test], N_MS_BANDS), 
                                    pan_tile_shape=(PAN_SIZE[train_val_test], PAN_SIZE[train_val_test], N_PAN_BANDS),
                                    sensor=sensor,
                                    band_selection=band_indices, 
                                    mean_correction=train_tiles_mean,
                                    cache_memory=False,
                                    cache_file=str(DATA_PATH_TILES + '/ds_' + EXPERIMENT + '-'
                                                   + train_val_test + '-' + sensor + '_filepath_cache'), 
                                    repeat=False, 
                                    shuffle=False, 
                                    shuffle_buffer_size=0, #SHUFFLE_BUFFER_SIZE[train_val_test], 
                                    include_file_paths=True)
    ds_val[sensor] = ds_val[sensor].get_dataset()
print(ds_val.keys())

dict_keys(['WV02', 'GE01'])


#### 8.1.2 Test set

In [15]:
train_val_test = 'test'
ds_test = {}
for sensor in SENSORS[train_val_test]:
    if sensor == 'WV02':
        band_indices = MS_BANDS_WV02_IDXS
    elif sensor == 'GE01':
        band_indices = MS_BANDS_GE01_IDXS
    ds_test[sensor] = GeotiffDataset(tiles_path=DATA_PATH_TILES_P[train_val_test], 
                                     batch_size=BATCH_SIZE[train_val_test], 
                                     ms_tile_shape=(MS_SIZE[train_val_test], MS_SIZE[train_val_test], N_MS_BANDS), 
                                     pan_tile_shape=(PAN_SIZE[train_val_test], PAN_SIZE[train_val_test], N_PAN_BANDS),
                                     sensor=sensor,
                                     band_selection=band_indices, 
                                     mean_correction=train_tiles_mean,
                                     cache_memory=False,
                                     cache_file=str(DATA_PATH_TILES + '/ds_' + EXPERIMENT + '-'
                                                    + train_val_test + '-' + sensor + '_filepath_cache'), 
                                     repeat=False, 
                                     shuffle=False, 
                                     shuffle_buffer_size=0)
    ds_test[sensor] = ds_test[sensor].get_dataset()
print(ds_test.keys())

dict_keys(['WV02', 'GE01'])


### 8.2 Evaluate last epoch

In [16]:
val_or_test = 'val'

# Computing Ma is 100x more time consuming than anything else. It is not interesting to measure this for pretraining
if METRIC_MA:
    PRE_GAN = ['gan']
else:
    PRE_GAN = ['pre', 'gan']
    
for pre_gan in PRE_GAN:
    for sensor in SENSORS[val_or_test]:
        if sensor == 'GE01':
            band_indices = MS_BANDS_GE01_IDXS
        elif sensor == 'WV02':
            band_indices = MS_BANDS_WV02_IDXS
        if pre_gan == 'pre':
            gan_model.G.load_weights(PRETRAIN_WEIGHTS_PATH)
        else:
            gan_model.G.load_weights(GAN_WEIGHTS_PATH)

        start = time.time()
        results_df = esrgan_evaluate(model=gan_model, 
                                     dataset=ds_val[sensor], 
                                     steps='all', 
                                     per_image=True, 
                                     write_csv=True,
                                     csv_path=str(LOGS_EXP_DIR + '/csv/' + 'final_epoch-' 
                                                  + pre_gan + '-' + val_or_test + '-' + sensor + '.csv'), 
                                     verbose=1
                                    )
        end = time.time()
        print(str((end - start) / 60), 'minutes')

Computed 8 images in  8.844409465789795 seconds
Last image: {'G_pixel_loss': 5.307237370288931e-05, 'G_perceptual_loss': 0.9389756321907043, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 39.94366455078125, 'SSIM': 0.9784864783287048}
Computed 8 images in  1.431025505065918 seconds
Last image: {'G_pixel_loss': 3.0085913749644533e-05, 'G_perceptual_loss': 0.6171954870223999, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 49.26082229614258, 'SSIM': 0.9942283034324646}
Computed 8 images in  1.4079484939575195 seconds
Last image: {'G_pixel_loss': 0.0001906752586364746, 'G_perceptual_loss': 3.803781032562256, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 30.10569190979004, 'SSIM': 0.8413370251655579}
Computed 8 images in  1.475562572479248 seconds
Last image: {'G_pixel_loss': 0.00013560103252530098, 'G_perceptual_loss': 2.2810888290405273, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, '

Computed 8 images in  1.4472613334655762 seconds
Last image: {'G_pixel_loss': 2.3990078261704184e-05, 'G_perceptual_loss': 0.5296525955200195, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 52.15961456298828, 'SSIM': 0.9985913038253784}
Computed 8 images in  1.4573071002960205 seconds
Last image: {'G_pixel_loss': 0.0001711587974568829, 'G_perceptual_loss': 2.9770240783691406, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 32.09608840942383, 'SSIM': 0.8269665241241455}
Computed 8 images in  1.4371676445007324 seconds
Last image: {'G_pixel_loss': 9.397071698913351e-05, 'G_perceptual_loss': 1.6402909755706787, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 38.85546875, 'SSIM': 0.9056259989738464}
Computed 8 images in  1.343813180923462 seconds
Last image: {'G_pixel_loss': 7.072593143675476e-05, 'G_perceptual_loss': 1.354645848274231, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR'

Computed 8 images in  1.4297990798950195 seconds
Last image: {'G_pixel_loss': 0.00011114142398582771, 'G_perceptual_loss': 2.810624361038208, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 35.04550552368164, 'SSIM': 0.8917247653007507}
Computed 8 images in  1.3848686218261719 seconds
Last image: {'G_pixel_loss': 8.149073255481198e-05, 'G_perceptual_loss': 1.0526269674301147, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 39.52658462524414, 'SSIM': 0.9272655248641968}
Computed 8 images in  1.3411898612976074 seconds
Last image: {'G_pixel_loss': 0.00014734144497197121, 'G_perceptual_loss': 2.325305461883545, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 33.26797866821289, 'SSIM': 0.8458905816078186}
Computed 8 images in  1.375037431716919 seconds
Last image: {'G_pixel_loss': 2.544830749684479e-05, 'G_perceptual_loss': 0.6147280335426331, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 

Computed 8 images in  1.418947458267212 seconds
Last image: {'G_pixel_loss': 6.081326137064025e-05, 'G_perceptual_loss': 1.2874161005020142, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 42.92145919799805, 'SSIM': 0.9326132535934448}
Computed 8 images in  1.4453160762786865 seconds
Last image: {'G_pixel_loss': 0.0005369509453885257, 'G_perceptual_loss': 2.2103424072265625, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 26.37232780456543, 'SSIM': 0.6359654664993286}
Computed 8 images in  1.444767951965332 seconds
Last image: {'G_pixel_loss': 0.0005522347637452185, 'G_perceptual_loss': 2.4044432640075684, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 25.575332641601562, 'SSIM': 0.7055471539497375}
Computed 8 images in  1.3802711963653564 seconds
Last image: {'G_pixel_loss': 0.000352579343598336, 'G_perceptual_loss': 3.4677841663360596, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, '

Computed 8 images in  1.3608927726745605 seconds
Last image: {'G_pixel_loss': 0.0004514747706707567, 'G_perceptual_loss': 3.3894810676574707, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 26.251527786254883, 'SSIM': 0.5458608269691467}
Computed 8 images in  1.4969823360443115 seconds
Last image: {'G_pixel_loss': 9.387889440404251e-05, 'G_perceptual_loss': 1.0735965967178345, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 38.41284942626953, 'SSIM': 0.9252984523773193}
Computed 8 images in  1.3948414325714111 seconds
Last image: {'G_pixel_loss': 0.0004906075191684067, 'G_perceptual_loss': 1.0264456272125244, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 27.637351989746094, 'SSIM': 0.7678722143173218}
Computed 8 images in  1.339179277420044 seconds
Last image: {'G_pixel_loss': 0.000480004062410444, 'G_perceptual_loss': 0.8911730051040649, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0,

...

Computed 8 images in  1.4455115795135498 seconds
Last image: {'G_pixel_loss': 0.00012191671703476459, 'G_perceptual_loss': 2.7002668380737305, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 35.33415603637695, 'SSIM': 0.9384435415267944}
Computed 8 images in  1.447676181793213 seconds
Last image: {'G_pixel_loss': 0.00015688766143284738, 'G_perceptual_loss': 3.307021379470825, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 32.349876403808594, 'SSIM': 0.8814144134521484}
Computed 8 images in  1.335512399673462 seconds
Last image: {'G_pixel_loss': 5.8343150158179924e-05, 'G_perceptual_loss': 1.1522924900054932, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 41.673912048339844, 'SSIM': 0.973554790019989}
Computed 8 images in  1.3349599838256836 seconds
Last image: {'G_pixel_loss': 0.0002051552146440372, 'G_perceptual_loss': 4.205632209777832, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0,

Computed 8 images in  1.3719820976257324 seconds
Last image: {'G_pixel_loss': 0.0002676736912690103, 'G_perceptual_loss': 4.934164047241211, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 28.669811248779297, 'SSIM': 0.6545112133026123}
Computed 8 images in  1.3501689434051514 seconds
Last image: {'G_pixel_loss': 0.00019677856471389532, 'G_perceptual_loss': 1.194059133529663, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 31.528465270996094, 'SSIM': 0.9580486416816711}
Computed 8 images in  1.447584629058838 seconds
Last image: {'G_pixel_loss': 0.00017893181939143687, 'G_perceptual_loss': 0.5475362539291382, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 35.24466323852539, 'SSIM': 0.9682718515396118}
Computed 8 images in  1.366713047027588 seconds
Last image: {'G_pixel_loss': 0.00017535492952447385, 'G_perceptual_loss': 1.4611636400222778, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0

Computed 8 images in  1.4367425441741943 seconds
Last image: {'G_pixel_loss': 0.000357142387656495, 'G_perceptual_loss': 5.498673439025879, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 27.348548889160156, 'SSIM': 0.5036522746086121}
Computed 8 images in  1.450038194656372 seconds
Last image: {'G_pixel_loss': 0.0005980923306196928, 'G_perceptual_loss': 1.8348712921142578, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 24.741395950317383, 'SSIM': 0.7820049524307251}
Computed 8 images in  1.4020743370056152 seconds
Last image: {'G_pixel_loss': 0.00045685641816817224, 'G_perceptual_loss': 0.8298488855361938, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 28.343778610229492, 'SSIM': 0.8834102153778076}
Computed 8 images in  1.445342779159546 seconds
Last image: {'G_pixel_loss': 0.0005000198725610971, 'G_perceptual_loss': 0.6921590566635132, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0,

Computed 8 images in  1.368943691253662 seconds
Last image: {'G_pixel_loss': 0.0005340799107216299, 'G_perceptual_loss': 1.2851080894470215, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 26.108434677124023, 'SSIM': 0.8127384781837463}
Computed 8 images in  1.4427542686462402 seconds
Last image: {'G_pixel_loss': 0.0005534264491870999, 'G_perceptual_loss': 2.7701148986816406, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 25.31599235534668, 'SSIM': 0.8070273399353027}
Computed 8 images in  1.3664708137512207 seconds
Last image: {'G_pixel_loss': 0.0004495455650612712, 'G_perceptual_loss': 0.7745421528816223, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 28.489463806152344, 'SSIM': 0.8898085355758667}
Computed 8 images in  1.4049737453460693 seconds
Last image: {'G_pixel_loss': 0.0006737100193277001, 'G_perceptual_loss': 3.824087142944336, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0,

NameError: name 'GAN_WEIGHTS_PATH' is not defined

### 8.3 Evaluate every kth epoch

### 8.4 Comparison plots